In [1]:
import numpy as np
from numpy import pi
import qiskit
#from qiskit import ClassicalRegister,QuantumRegister,QuantumCircuit,execute, assemble, Aer, IBMQ
from qiskit import *
from qiskit.visualization import plot_histogram, plot_bloch_vector
from qiskit.circuit.library.standard_gates import CU3Gate

In [2]:
class newNode: 
    def __init__(self,data): 
        self.data = data
        self.left = self.right = None

In [3]:
def create_binary_tree(n,i,root):
    limit = n - 1        
    if i < limit:
        temp = newNode(0)
        root = temp        
        root.left = create_binary_tree(n,2*i+1,root.left)
        root.right = create_binary_tree(n,2*i+2,root.right)
    return root 

In [4]:
def check_value(n):
    if n < 0 :
        return -np.power(n,2)
    else: 
        return np.power(n,2)

In [5]:
def insert_leaves(arr,root,i):
    global index
    index = i
    a = 0
    if (not root):
        return
    if (not root.left and 
        not root.right):
        root.left = newNode(check_value(arr[index]))
        if (index + 1) < len(arr):
            root.right = newNode(check_value(arr[index + 1]))
        if (index + 2) < len(arr):
            index = index + 2
        else: 
            index = index + 1
        return a 
    
    if root.left :
        a = insert_leaves(arr,root.left,index)
    if root.right :
        a = insert_leaves(arr,root.right,index)       

In [19]:
def fill_tree(root):
    a = 0
    if (not root):
        return
    if (root.left):
        if (not root.left.left):
            root.data = abs(root.left.data) + abs(root.right.data)
        else:
            a = fill_tree(root.left)
    if (root.right) :
        if (root.right.left):
            root.right.data = abs(root.right.left.data) + abs(root.right.right.data)
            a = fill_tree(root.right)
        else:   
            return a
    root.data = abs(root.left.data) + abs(root.right.data)

In [7]:
def normalized_array(arr):
    sum_square = 0
    for i in range (len(arr)):
        sum_square += np.power(arr[i],2)
    for i in range (len(arr)):
        arr[i] =arr[i] / np.sqrt(sum_square)
    return arr

In [26]:
def calculate_theta_phi(arr):
    print(arr)
    # We only use U3 gate to construct state from state |0> therefore, the state after applying U3 gate is: [cos(theta/2) e^(i*phi)*sin(theta/2)]
    # We can easily find out theta using arccos
    # Because the preference matrix entries are real number, therefore the imaginary part of e^(i*phi) affect only the phase (sign) of the state .
    # We have phi = 0 if sign is "+", phi = pi if sign is "-"
    
    angle = np.zeros(2)
    theta = 2 * np.arccos(arr[0])
    if(arr[1] > 0):
        phi = 0 
    if(arr[1] < 0):
        phi = pi
    angle[0] = theta
    angle[1] = phi
    
    return angle

In [9]:
def condition_gate(qc, i, n, num_qubit):
    binary_form = np.binary_repr(i,n)    
    for i in range (len(binary_form)):
        if (binary_form[i] == '0'):
            qc.x(num_qubit - 1 - i)

In [10]:
def arr_of_CU3(layer, num_qubit):
    arr = []
    for i in range (layer):
        arr.append(int(num_qubit - 1 - i))
    return arr

In [28]:
def height(node):
    if node is None:
        return 0
    else :
        # Compute the height of each subtree 
        lheight = height(node.left)
        rheight = height(node.right)
 
        #Use the larger one
        if lheight > rheight :
            return lheight+1
        else:
            return rheight+1
        
def initialize_GivenLevel(root , level, layer, qc, num_qubit):
    if root is None:
        return
    if (layer == 1) and num_qubit > 1:
        
        initialize_state_0 = [np.sqrt(root.left.data), np.sqrt(root.right.data)]
        initialize_state_0 = calculate_theta_phi(initialize_state_0)
        qc.u3(initialize_state_0[0],initialize_state_0[1],0,num_qubit-1)
        
    if (layer == 1) and num_qubit == 1:
        
        initialize_state_0 = [np.sqrt(root.left.data), np.sqrt(root.right.data)]
        initialize_state_0 = calculate_theta_phi(initialize_state_0)
        qc.u3(initialize_state_0[0],initialize_state_0[1],0,0)
        
    if layer == 2:
        
        initialize_state_1 = [1/np.sqrt(root.left.data) * np.sqrt(root.left.left.data), 1/np.sqrt(root.left.data) * np.sqrt(root.left.right.data)]
        initialize_state_2 = [1/np.sqrt(root.right.data) * np.sqrt(root.right.left.data), 1/np.sqrt(root.right.data) * np.sqrt(root.right.right.data)]
        
        initialize_state_1 = calculate_theta_phi(initialize_state_1)
        initialize_state_2 = calculate_theta_phi(initialize_state_2)
        
        qc.cu3(initialize_state_2[0],initialize_state_2[1],0,num_qubit-1,num_qubit-2)

        qc.x(num_qubit-1)

        qc.cu3(initialize_state_1[0],initialize_state_1[1],0,num_qubit-1,num_qubit-2)

        qc.x(num_qubit-1)
        
    if layer > 2 : 
        global i
        if level == layer :
            i = 0

        if level == 1:

            initialize_state = [(1/np.sqrt(root.data))* np.sqrt(root.left.data), (1/np.sqrt(root.data)) * np.sqrt(root.right.data)]
            #print (initialize_state)
            initialize_state = calculate_theta_phi(initialize_state)
            
            qc.barrier()
            condition_gate(qc, i, layer - 1, num_qubit)

            CU3 = CU3Gate(initialize_state[0],initialize_state[1],0).control(layer-2)
            
            arr = arr_of_CU3(layer, num_qubit)           
            qc.append(CU3 , arr)

            condition_gate(qc, i, layer - 1, num_qubit)
            qc.barrier()

            i += 1

        elif level > 1 :

            initialize_GivenLevel(root.left , level-1, layer, qc, num_qubit)
            initialize_GivenLevel(root.right , level-1, layer, qc, num_qubit)
        
def initialize_LevelOrder(root, qc, num_qubit):
    h = height(root)
    for i in range(1, h+1):
        initialize_GivenLevel(root, i-1, i-1, qc, num_qubit)

In [36]:
arr = [-0.1, 0.2, 0.15, 0.1, 0.05, 0.2, 0.17, 0.03, ]

n = len(arr)
normalized_arr = normalized_array(arr)

root = newNode(0)
root = create_binary_tree(n,0,root)
insert_leaves(normalized_arr,root,0)
fill_tree(root)
print(root.left.left.left.data)
print(root.data)

-0.06459948320413436
0.9999999999999998


In [37]:
num_qubit = int(np.log2(n))
qc = QuantumCircuit(num_qubit)

# initialize_state_0 = [np.sqrt(root.left.data), np.sqrt(root.right.data)]
# initialize_state_0 = calculate_theta_phi(initialize_state_0)

# qc.u3(initialize_state_0[0],initialize_state_0[1],0,1)

# initialize_state_1 = [1/np.sqrt(root.left.data) * np.sqrt(root.left.left.data), 1/np.sqrt(root.left.data) * np.sqrt(root.left.right.data)]
# initialize_state_2 = [1/np.sqrt(root.right.data) * np.sqrt(root.right.left.data), 1/np.sqrt(root.right.data) * np.sqrt(root.right.right.data)]

# initialize_state_1 = calculate_theta_phi(initialize_state_1)
# initialize_state_2 = calculate_theta_phi(initialize_state_2)

# CU3 = CU3Gate(initialize_state_2[0],initialize_state_2[1],0)
# qc.append(CU3,[1,0])

# qc.x(1)

# qc.cu3(initialize_state_1[0],initialize_state_1[1],0,1,0)

# qc.x(1)
initialize_LevelOrder(root, qc, num_qubit)

qc = transpile(qc, basis_gates=['u1','u2','u3','cx','x','z'])

qobj = assemble(qc)     # Create a Qobj from the circuit for the simulator to run
svsim = Aer.get_backend('statevector_simulator') # Tell Qiskit how to simulate our circuit
result = svsim.run(qobj).result()
out_state = result.get_statevector()
#qc.draw()
print(out_state) # Display the output state vector
#results = svsim.run(qobj).result().get_counts()
#plot_histogram(results)



[0.7300313256526109, 0.6834136840639726]
[0.778498944161523, 0.6276459144608478]
[0.7666997407021915, 0.642005847019474]
[nan, 0.8944271909999159]
[0.8320502943378436, 0.5547001962252291]
[0.24253562503633297, 0.9701425001453319]
[0.9847835588179368, 0.17378533390904763]


C:\Users\trongduong\Anaconda3\envs\QML\lib\site-packages\ipykernel_launcher.py:53: RuntimeWarning: invalid value encountered in sqrt
C:\Users\trongduong\Anaconda3\envs\QML\lib\site-packages\qiskit\transpiler\passes\optimization\optimize_1q_gates.py:163: RuntimeWarning: invalid value encountered in remainder
  (2 * np.pi))) < self.eps and right_name != "u1":
C:\Users\trongduong\Anaconda3\envs\QML\lib\site-packages\qiskit\transpiler\passes\optimization\optimize_1q_gates.py:175: RuntimeWarning: invalid value encountered in remainder
  2 * np.pi)) < self.eps:
C:\Users\trongduong\Anaconda3\envs\QML\lib\site-packages\qiskit\transpiler\passes\optimization\optimize_1q_gates.py:185: RuntimeWarning: invalid value encountered in remainder
  2 * np.pi)) < self.eps:


ValueError: array must not contain infs or NaNs

In [35]:
arr / np.linalg.norm(arr)

array([0.17972129, 0.35944258, 0.26958193, 0.17972129, 0.08986064,
       0.35944258, 0.30552619, 0.05391639, 0.17972129, 0.35944258,
       0.26958193, 0.17972129, 0.08986064, 0.35944258, 0.30552619,
       0.05391639])